In [60]:
import requests
import datetime
import pandas as pd
import json
import numpy as np

In [61]:
def crawl_price(security_id):
    start = int(datetime.datetime.now().timestamp()) - 86400 * 6 #設個緩衝天數，以免遇到各國股市有休息的。
    now = int(datetime.datetime.now().timestamp()) + 86400
    url = "https://query1.finance.yahoo.com/v8/finance/chart/" + security_id + "?period1=" + str(start) + "&period2=" + str(now) + "&interval=1d&events=history&=hP2rOschxO0"
    response = requests.get(url)
    data = json.loads(response.text)
    df = pd.DataFrame(data['chart']['result'][0]['indicators']['quote'][0], index=pd.to_datetime(np.array(data['chart']['result'][0]['timestamp'])*1000*1000*1000))
    return df
def crawl_price_fx(security_id):
    start = int(datetime.datetime.now().timestamp()) - 86400 * 365
    now = int(datetime.datetime.now().timestamp()) + 86400
    url = "https://query1.finance.yahoo.com/v8/finance/chart/" + security_id + "?period1=" + str(start) + "&period2=" + str(now) + "&interval=1d&events=history&=hP2rOschxO0"
    response = requests.get(url)
    data = json.loads(response.text)
    df = pd.DataFrame(data['chart']['result'][0]['indicators']['quote'][0], index=pd.to_datetime(np.array(data['chart']['result'][0]['timestamp'])*1000*1000*1000))
    return df

In [70]:
"""
主要金融市場指數代碼表
"^TWII" 台灣加權指數
"^DJI" 道瓊工業指數
"^GSPC" S&P500
"^IXIC" NASDAQ
"^RUT" Russell2000
"^VIX" VIX
"000001.SS" 上證指數
"^HSI" 恆生指數 
"^N225" 日經指數
"^KS11" 韓國綜合指數
"^FTSE" 富時100指數
"^GDAXI" 德國DAX指數
"^FCHI" 法國巴黎CAC40指數
"^STI" 新加坡海峽時報指數
"^KLSE" 馬來西亞吉隆坡綜合指數
"^BSESN" 印度SENSEX指數
"^AORD" 澳洲股市指數
"""
globalIndexCodeList = ["^TWII", "^DJI", "^GSPC", "^IXIC", "^RUT", "^VIX", "000001.SS", "^HSI", "^N225", "^KS11", "^FTSE", "^GDAXI", "^FCHI", "^STI", "^KLSE", "^BSESN", "^AORD"]
globalIndexNameList = ["台灣加權指數", "道瓊工業指數", "S&P500", "NASDAQ", "Russell2000", "VIX", "上證指數", "恆生指數", "日經指數", "韓國綜合指數"
                       , "富時100指數", "德國DAX指數", "法國巴黎CAC40指數", "新加坡海峽時報指數", "馬來西亞吉隆坡綜合指數", "印度SENSEX指數", "澳洲股市指數"]
globalIndexList = []
for i in globalIndexCodeList:
    df = crawl_price(i)
    df["指數名稱"] = globalIndexNameList[globalIndexCodeList.index(i)]
    df["報價時間"] = df.index
    df["開盤"] = [round(float(j),2) for j in df["open"]]
    df["最高"] = [round(float(j),2) for j in df["high"]]
    df["最低"] = [round(float(j),2) for j in df["low"]]
    df["指數成交"] = [round(float(j),2) for j in df["close"]]
    for k in range(len(df)):
        if k == 0:
            a = []
            b = []
            a.append("NA")
            b.append("NA")
        else:
            a.append(df["指數成交"][k] - df["指數成交"][k-1])
            b.append(round(((df["指數成交"][k] - df["指數成交"][k-1])/df["指數成交"][k-1]), 4) *100)
    df["漲跌點數"] = a
    df["漲跌幅(%)"] = b
    df = df[["指數名稱", "報價時間", "開盤", "最高", "最低", "指數成交", "漲跌點數", "漲跌幅(%)"]]
    df.reset_index(drop = True, inplace = True)
    df.drop(index = range(len(df)-1), inplace=True)
    globalIndexList.append(df)
df_globalIndex = pd.concat(globalIndexList)
df_globalIndex.reset_index(drop = True, inplace = True)
# save file : df_globalIndex.to_csv('全球指數資料庫.csv', encoding="utf-8", index = 0)

"""
主要原物料指數代碼表
"GC=F" 黃金
"SI=F" 白銀
"PL=F" 白金
"HG=F" 銅
"CL=F" 西德州輕原油
"NG=F" 天然氣
"BZ=F" 布蘭特原油
"S=F" 黃豆
"""
commodityCodeList = ["GC=F", "SI=F", "PL=F", "HG=F", "CL=F", "NG=F", "BZ=F", "S=F"]
commodityNameList = ["黃金", "白銀", "白金", "銅", "西德州輕原油", "天然氣", "布蘭特原油", "黃豆"]
commodityIndexList = []
for i in commodityCodeList:
    df = crawl_price(i)
    df["商品名稱"] = commodityNameList[commodityCodeList.index(i)]
    df["報價時間"] = df.index
    df["開盤"] = [round(float(j),2) for j in df["open"]]
    df["最高"] = [round(float(j),2) for j in df["high"]]
    df["最低"] = [round(float(j),2) for j in df["low"]]
    df["報價(USD計價)"] = [round(float(i),2) for i in df["close"]]
    for i in range(len(df)):
        if i == 0:
            a = []
            b = []
            a.append("NA")
            b.append("NA")
        else:
            a.append(df["報價(USD計價)"][i] - df["報價(USD計價)"][i-1])
            b.append(round(((df["報價(USD計價)"][i] - df["報價(USD計價)"][i-1])/df["報價(USD計價)"][i-1]), 4) *100)
    df["漲跌"] = a
    df["漲跌幅(%)"] = b
    df = df[["商品名稱", "報價時間", "開盤", "最高", "最低", "報價(USD計價)", "漲跌", "漲跌幅(%)"]]
    df.reset_index(drop = True, inplace = True)
    df.drop(index = range(len(df)-1), inplace=True)
    commodityIndexList.append(df)
df_commodityIndex = pd.concat(commodityIndexList)
df_commodityIndex.reset_index(drop = True, inplace = True)
# save file : df_commodityIndex.to_csv('原物料指數資料庫.csv', encoding="utf-8", index = 0)

"""
主要匯率代碼表
"DX-Y.NYB" 美元指數
"EURUSD=X" 歐元兌美元
"GBPUSD=X" 英鎊兌美元
"JPY=X" 美元兌日圓
"NZD=X" 美元兌紐幣
"CAD=X" 美元兌加幣
"AUD=X" 美元兌澳幣
"CNH=X" 美元兌離岸人民幣
"TWD=X" 美元兌台幣
"""
fxCodeList = ["DX-Y.NYB", "EURUSD=X", "GBPUSD=X", "JPY=X", "NZD=X", "CAD=X", "AUD=X", "TWD=X"]
fxNameList = ["美元指數", "歐元兌美元", "英鎊兌美元", "美元兌日圓", "美元兌紐幣", "美元兌加幣", "美元兌澳幣", "美元兌台幣"]
fxIndexList = []
for i in fxCodeList:
    df = crawl_price_fx(i)
    df["匯兌名稱"] = fxNameList[fxCodeList.index(i)]
    df["報價時間"] = df.index
    df["開盤"] = [round(float(j),2) for j in df["open"]]
    df["最高"] = [round(float(j),2) for j in df["high"]]
    df["最低"] = [round(float(j),2) for j in df["low"]]
    df["收盤"] = [round(float(i),2) for i in df["close"]]
    for i in range(len(df)):
        if i == 0:
            a = []
            b = []
            a.append("NA")
            b.append("NA")
        else:
            a.append(df["收盤"][i] - df["收盤"][i-1])
            b.append(round(((df["收盤"][i] - df["收盤"][i-1])/df["收盤"][i-1]), 4) *100)
    df["漲跌"] = a
    df["漲跌幅(%)"] = b
    df = df[["匯兌名稱", "報價時間", "開盤", "最高", "最低", "收盤", "漲跌", "漲跌幅(%)"]]
    df.reset_index(drop = True, inplace = True)
    fxIndexList.append(df)
df_fxIndex = pd.concat(fxIndexList)
df_fxIndex.reset_index(drop = True, inplace = True)
# save file : df_fxIndex.to_csv('匯率資料庫.csv', encoding="utf-8", index = 0)